# Neural Network for Image Classification with Numpy

In this notebook, I'll be making a Neural network for image classification on the MNIST Dataset. A dataset containing handwritten digits of 60,000 training examples and 10,000 testing examples.

Goals:
- [Loading Data and Initializing Parameters](#loading-the-data)
- [Forward Propagation](#forward-propagation)
- [Backward Propagation](#backward-propagation)
- [Testing the model](#testing-the-model)